<a href="https://colab.research.google.com/github/Sanskar6877/BTP/blob/main/cross_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_text
import tensorflow_text as text
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
import json

# Choose the file from your local machine
uploaded = files.upload()

# Read the JSON file with explicit encoding specification
json_filename = list(uploaded.keys())[0]
with open(json_filename, 'r', encoding='utf-8', errors='replace') as file:
    data = json.load(file)

# Convert the loaded JSON data to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index')

# Create a binary 'hate' column based on the 'label_off' column
df['hate'] = df['label'].apply(lambda x: 0 if x == 'normal' else 1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['post'], df['hate'], stratify=df['hate'])

# BERT preprocessor and encoder
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name='Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='Dense')(dropout)

# Build and compile the model
model = tf.keras.Model(inputs=[text_input], outputs=[outputs])

Metrics = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=Metrics)

# Train the model
history = model.fit(X_train, y_train, epochs=10)

# Evaluate the model
model.evaluate(X_test, y_test)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.0 MB/s eta 0:00:00
Mounted at /content/drive


Saving hateX_test.json to hateX_test.json
Epoch 1/10
46/46 [==============================] - 28s 308ms/step - loss: 0.6839 - accuracy: 0.5787 - precision: 0.6021 - recall: 0.8540
Epoch 2/10
46/46 [==============================] - 14s 307ms/step - loss: 0.6838 - accuracy: 0.5856 - precision: 0.6110 - recall: 0.8294
Epoch 3/10
46/46 [==============================] - 14s 310ms/step - loss: 0.6694 - accuracy: 0.6050 - precision: 0.6164 - recall: 0.8843
Epoch 4/10
46/46 [==============================] - 14s 312ms/step - loss: 0.6672 - accuracy: 0.5981 - precision: 0.6135 - recall: 0.8715
Epoch 5/10
46/46 [==============================] - 14s 314ms/step - loss: 0.6629 - accuracy: 0.6064 - precision: 0.6231 - recall: 0.8516
Epoch 6/10
46/46 [==============================] - 14s 315ms/step - loss: 0.6620 - accuracy: 0.6036 - precision: 0.6166 - recall: 0.8773
Epoch 7/10
46/46 [==============================] - 15s 318ms/step - loss: 0.6577 - accuracy: 0.6071 - precision: 0.6240 - recall:

[0.6308196187019348,
 0.6278586387634277,
 0.6258823275566101,
 0.9300699234008789]